In [2]:
import keras.backend as K
import tensorflow as tf
import numpy as np


CAM_INTR_COLOR = K.constant([[1395.749023, 0, 935.732544],
                            [0, 1395.749268, 540.681030],
                            [0, 0, 1]])

skel_xyz = tf.Variable(np.ones((21, 3), dtype=np.float32))
pred_offset = tf.Variable(np.ones((21, 3), dtype=np.float32))

init_op = tf.initialize_all_variables()
# print(CAM_INTR_COLOR[1:, :])
with tf.Session() as sess:
    sess.run(init_op)
    skel_xyz_intr = K.transpose(K.dot(CAM_INTR_COLOR, K.transpose(skel_xyz)))
#     print(tf.math.divide(skel_xyz_intr[:, 0],skel_xyz_intr[:, 2])
    skel_uvd = tf.stack([skel_xyz_intr[:, 0]/skel_xyz_intr[:, 2], 
                         skel_xyz_intr[:, 1]/skel_xyz_intr[:, 2], 
                         skel_xyz_intr[:, 2]], 
                         axis = 1)
#     print(skel_uvd)
#     print(pred_offset)
#     print(pred_offset[0, :])
    pred_hand_pose_root = tf.expand_dims(tf.stack([pred_offset[0, 0], pred_offset[0, 1]+1, pred_offset[0, 2]+2]), 0)
    pred_hand_pose_rest = tf.map_fn(lambda x: [x[0], x[1]+1, x[2]+2], tf.unstack(pred_offset[1:, :], axis=1))
#     print(pred_hand_pose_root)
    pred_hand_pose_rest_test = K.transpose(tf.stack(pred_hand_pose_rest))
    print(pred_hand_pose_rest_test.eval())
#     pred_hand_pose = tf.concat([pred_hand_pose_root, pred_hand_pose_rest], axis=0)
#     print(pred_hand_pose)

Using TensorFlow backend.


Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]


In [52]:
with tf.Session() as sess:
    index = K.arange(845)
    unravel_index = K.transpose(tf.unravel_index(index, (13, 13, 5)))
    print(unravel_index.eval())

[[ 0  0  0]
 [ 0  0  1]
 [ 0  0  2]
 ...
 [12 12  2]
 [12 12  3]
 [12 12  4]]


In [ ]:
# def hpluso_loss(y_gt, y_pred):
#     # y_pred = K.reshape(y_pred, (-1, W, H, D, 64))
#     conf_loss = 0
#     hpe_loss = K.zeros([1])
#     # convert to image space
#     # y_uvd_gt = camcoord2uvd(y_gt)

#     # min_u = K.min(y_uvd_gt[:, 0])
#     # max_u = K.max(y_uvd_gt[:, 0])
#     # min_v = K.min(y_uvd_gt[:, 1])
#     # max_v = K.max(y_uvd_gt[:, 1])
#     # min_z = K.min(y_uvd_gt[:, 2])
#     # max_z = K.max(y_uvd_gt[:, 2])
#     # hand_cell_range_u = list(range(int(min_u/C_u), tf.math.ceil(max_u/C_u)))
#     # hand_cell_range_v = list(range(int(min_v/C_v), tf.math.ceil(max_v/C_v)))
#     # hand_cell_range_z = list(range(int(min_z/C_z), tf.math.ceil(max_z/C_z)))

#     # for u in range(0, W):
#     #     for v in range(0, H):
#     #         for z in range(0, D):

#     #             #hand pose
#     #             pred_offset = y_pred[u, v, z, :63]
#     #             pred_offset = K.reshape(pred_offset, (21,3))
#     #             pred_hand_pose = K.zeros(pred_offset.shape)

#     #             pred_hand_pose_root = tf.expand_dims(tf.stack([ K.sigmoid(pred_offset[0, 0])+u,  K.sigmoid(pred_offset[0, 1])+v,  K.sigmoid(pred_offset[0, 2])+z]), 0)
#     #             pred_hand_pose_rest = tf.map_fn(lambda x: tf.stack([x[0]+u, x[1]+v, x[2]+z])+z, pred_offset[1:, :])
#     #             pred_hand_pose = tf.concat([pred_hand_pose_root, pred_hand_pose_rest], axis=0)

#     #             # #confidence
#     #             # print(pred_hand_pose[:, :2])
#     #             # print(y_gt)
#     #             # print(y_uvd_gt[:, :2])
#     #             # DT_uv = l2_dist(pred_hand_pose[:, :2], y_uvd_gt[:, :2])
#     #             # if DT_uv < d_th:
#     #             #     conf_uv = K.exp(a*(1-(DT_uv/d_th)))
#     #             # else:
#     #             #     conf_uv = 0
#     #             # DT_z = l2_dist(pred_hand_pose[:, 2], skel_uvd_gt[:, 2])
#     #             # if DT_z < d_th:
#     #             #     conf_z = K.exp(a*(1-(DT_z/d_th)))
#     #             # else:
#     #             #     conf_z = 0
#     #             # conf = 0.5*conf_uv + 0.5*conf_z

#     #             #calc loss
#     #             # pred_hand_pose[:, 0] = C_u*pred_hand_pose[:, 0]
#     #             # pred_hand_pose[:, 1] = C_v*pred_hand_pose[:, 1]
#     #             # pred_hand_pose[:, :2] = C_z*pred_hand_pose[:, :2]*pred_hand_pose[:, 2:]
#     #             # pred_poses_xyz = K.transpose(K.dot(INV_CAM_INTR_COLOR, K.transpose(pred_hand_pose)))
#     #             # print(pred_poses_xyz)
#     #             # # conf_loss += l2_dist(y_pred[u, v, z, 63], conf)
#     #             # hpe_loss += l2_dist(pred_poses_xyz, y_gt)

#     return hpe_loss


# def camcoord2uvd(skel_xyz):
#     skel_xyz_intr = K.transpose(K.dot(CAM_INTR_COLOR, K.transpose(skel_xyz)))
#     skel_uvd = tf.stack([skel_xyz_intr[:, 0]/skel_xyz_intr[:, 2],
#                          skel_xyz_intr[:, 1]/skel_xyz_intr[:, 2],
#                          skel_xyz_intr[:, 2]], axis = 1)

#     return skel_uvd

In [54]:
np.zeros((845)).shape

(845,)

In [32]:
def l2_dist(a, b, ax=-1):
    return K.sqrt(K.sum(K.square(a - b), axis=ax, keepdims=True))

with tf.Session() as sess:
    index = K.constant(np.ones((21,3)))
    index_2 = K.constant(np.zeros((21,3)))
#     print(K.shape(index).eval())
#     print(l2_dist(index, index_2, -1).eval())
    test = K.arange(50)
#     print(test.eval())
#     zz = tf.where(K.greater(test, 4))
#     print(zz.eval())
#     print(tf.gather_nd(test, zz).eval())
    print(K.cast(K.greater(test,4).eval(), dtype=tf.float32).eval())
#     print(test[zz].eval())

[0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]


In [ ]:
# if __name__ == '__main__':
#     train_pairs = []
#     test_pairs = []
#     with open(os.path.join(constants.DATASET_DIR, 'data_split_action_recognition.txt')) as f:
#         cur_split = 'Training'
#         lines = f.readlines()
#         for l in lines:
#             words = l.split()
#             if(words[0] == 'Training' or words[0] == 'Test'):
#                 cur_split = words[0]
#             else:
#                 path = l.split()[0]
#                 full_path = os.path.join(constants.IMG_DIR, path, 'color')
#                 len_frame_idx = len([x for x in os.listdir(full_path)
#                                     if os.path.join(full_path, x)])
#                 skeleton_path = os.path.join(constants.SKEL_DIR, path, 'skeleton.txt')
#                 skeleton_vals = np.loadtxt(skeleton_path)
#                 for i in range(len_frame_idx):
#                     img_path = os.path.join(constants.IMG_DIR, path, 'color', 'color_%04d.jpeg' %i)
#                     skel_xyz = skeleton_vals[:, 1:].reshape(skeleton_vals.shape[0], 21, 3)[i]
#                     data_pair = (img_path, skel_xyz)
#                     if cur_split == 'Training':
#                         train_pairs.append(data_pair)
#                     else:
#                         test_pairs.append(data_pair)

#     generator = DataGenerator(train_pairs)
#     from matplotlib import pyplot as plt
#     data = generator[0]
#     fig, ax = plt.subplots()
#     ax.imshow(data[0][0])
#     skel_proj = utils.xyz2uvd(np.reshape(data[1][0], (21,3)), camcoord=True)
#     utils.visualize_joints_2d(ax, skel_proj[constants.REORDER], joint_idxs=False)
#     plt.show()